## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo Rides</strong>

<strong>3. Pooled Rides</strong>

## Load ExMAS and MaaSSim

In [489]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
#from MaaSSim.performance import kpi_pax_pricing
#from MaaSSim.performance import kpi_veh_pricing


import logging
import matplotlib.pyplot as plt


import pandas as pd
import ExMAS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Delft, Netherlands

In [490]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 4 # 6 minutes hour of simulation
params.nP = 100 # reuqests (and passengers)
params.nV = 20 # vehicles


## Parameters for ExMAS

In [491]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.0005
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True


# params.kpi = 1 (Profit Maximazation)
# params.kpi = 2 (Solo Rides)
# params.kpi = 3 (Pooled Rides)

In [492]:
params.kpi = 3

In [493]:
#inData = ExMAS.main(inData, params.shareability) # create shareability graph (ExMAS) 
inData = ExMAS.main(inData, params.shareability, plot = False)

31-07-23 20:27:10-INFO-Initializing pairwise trip shareability between 100 and 100 trips.
31-07-23 20:27:10-INFO-creating combinations
31-07-23 20:27:10-INFO-9900	 nR*(nR-1)
31-07-23 20:27:11-INFO-Reduction of feasible pairs by 99.89%
31-07-23 20:27:11-INFO-Degree 2 	Completed
31-07-23 20:27:11-INFO-trips to extend at degree 2 : 36
31-07-23 20:27:11-INFO-At degree 2 feasible extensions found out of 0 searched
31-07-23 20:27:11-INFO-Degree 3 	Completed
31-07-23 20:27:11-INFO-No more trips to exted at degree 3


In [494]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

31-07-23 20:27:11-INFO-Matching 100 trips to 136 rides in order to min u_veh
31-07-23 20:27:11-INFO-Problem solution: Optimal. 
Total costs for single trips:         25,410 
reduced by matching to:               24,018


In [495]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],6.243500,337,1,[6.2435],"[0, 337]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44722228, 4159110115]","[None, 0, 0]",node time req_id od 0 ...,337,3.37,5.064000,1.012800,4.0512
1,[1],5.745500,310,1,[5.7455],"[10, 310]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1448535729, 44814377]","[None, 1, 1]",node time req_id od 0 ...,310,3.10,4.660500,0.932100,3.7284
2,[2],1.429000,77,1,[1.429],"[127, 77]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44785094, 44771654]","[None, 2, 2]",node time req_id od 0 NaN...,77,0.77,1.159500,0.231900,0.9276
3,[3],5.499000,297,1,[5.4990000000000006],"[297, 297]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44770870, 385474972]","[None, 3, 3]",node time req_id od 0 N...,297,2.97,4.459500,0.891900,3.5676
4,[4],9.411500,508,1,[9.4115],"[368, 508]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 6597272889, 44740049]","[None, 4, 4]",node time req_id od 0 ...,508,5.08,7.633500,1.526700,6.1068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,"[78, 81]",10.960825,739,21,"[8.82845, 2.132375]","[10364.0, 271, 130, 338]","[78, 81]","[81, 78]",2,131,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44735538, 3377240325, 1558185438, 44847...","[None, 78, 81, 81, 78]",node time req_id od 0 ...,739,7.39,6.870375,1.374075,5.4963
132,"[19, 24]",8.770250,372,21,"[5.7583, 3.0119499999999997]","[2673.0, 187, 165, 20]","[19, 24]","[24, 19]",2,132,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 792053933, 3377240340, 44855652, 44852053]","[None, 19, 24, 24, 19]",node time req_id od 0 ...,372,3.72,5.517000,1.103400,4.4136
133,"[45, 47]",6.562900,419,21,"[5.55855, 1.00435]","[5528.5, 258, 55, 106]","[45, 47]","[47, 45]",2,133,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 3684853380, 2341955079, 44818486, 21626...","[None, 45, 47, 47, 45]",node time req_id od 0 ...,419,4.19,4.137750,0.827550,3.3102
134,"[79, 82]",9.960400,458,21,"[5.7467500000000005, 4.2136499999999995]","[10525.5, 169, 261, 28]","[79, 82]","[82, 79]",2,134,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44831894, 549099463, 2598004622, 434904...","[None, 79, 82, 82, 79]",node time req_id od 0 ...,458,4.58,6.511500,1.302300,5.2092


In [496]:
sim = simulate(params = params, inData = inData,  logger_level = logging.CRITICAL) # simulate

31-07-23 20:27:12-INFO-Matching 100 trips to 136 rides in order to min u_veh
31-07-23 20:27:12-INFO-Problem solution: Optimal. 
Total costs for single trips:         25,410 
reduced by matching to:               24,018
0.0005
31-07-23 20:27:14-CRITICAL-this is request 0 with [0] available rides.
31-07-23 20:27:14-CRITICAL-ride 0 available [1]
31-07-23 20:27:14-CRITICAL-this is request 0 with [0] still available rides.
31-07-23 20:27:14-CRITICAL-vehicle 1 has 1 choices
0.0005
31-07-23 20:27:14-CRITICAL-this is request 1 with [1, 108] available rides.
31-07-23 20:27:14-CRITICAL-ride 1 available [1]
31-07-23 20:27:14-CRITICAL-ride 108 available [1, 0]
31-07-23 20:27:14-CRITICAL-this is request 1 with [1, 108] still available rides.
31-07-23 20:27:14-CRITICAL-vehicle 17 has 2 choices
0.0005
31-07-23 20:27:14-CRITICAL-this is request 2 with [2] available rides.
31-07-23 20:27:14-CRITICAL-ride 2 available [1]
31-07-23 20:27:14-CRITICAL-this is request 2 with [2] still available rides.
31-07-

In [497]:
sim.res[0].pax_exp

,ACCEPTS_OFFER,ARRIVES_AT_DROPOFF,ARRIVES_AT_PICKUP,DEPARTS_FROM_PICKUP,LOSES_PATIENCE,MEETS_DRIVER_AT_PICKUP,REQUESTS_RIDE,SETS_OFF_FOR_DEST,STARTS_DAY,RECEIVES_OFFER,IS_REJECTED_BY_VEHICLE,REJECTS_OFFER,ARRIVES_AT_DEST,PREFERS_OTHER_SERVICE,TRAVEL,WAIT,OPERATIONS,LOST_PATIENCE
pax,,,,,,,,,,,,,,,,,,
0,15.0,337.0,20.0,30.0,0.0,343.0,0.0,10.0,0,0,0,0,0,0,337.0,343.0,55.0,False
1,15.0,310.0,20.0,30.0,0.0,7.0,10.0,10.0,0,0,0,0,0,0,310.0,7.0,55.0,False
2,15.0,77.0,20.0,30.0,0.0,52.0,127.0,10.0,0,0,0,0,0,0,77.0,52.0,55.0,False
3,15.0,297.0,20.0,30.0,0.0,50.0,297.0,10.0,0,0,0,0,0,0,297.0,50.0,55.0,False
4,15.0,508.0,20.0,30.0,0.0,120.0,368.0,10.0,0,0,0,0,0,0,508.0,120.0,55.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.0,188.0,20.0,30.0,0.0,0.0,13604.0,10.0,0,0,0,0,0,0,188.0,0.0,55.0,False
96,15.0,144.0,20.0,30.0,0.0,0.0,13698.0,10.0,0,0,0,0,0,0,144.0,0.0,55.0,False
97,15.0,121.0,20.0,30.0,0.0,53.0,13722.0,10.0,0,0,0,0,0,0,121.0,53.0,55.0,False


# Service rate 

In [498]:
Fulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[False]
unFulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[True]
service_rate = Fulfilled_requests/(Fulfilled_requests+unFulfilled_requests)
print('service_rate = ', service_rate*100, '%')

service_rate =  76.0 %


In [499]:
#sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv')
#sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv')

In [500]:
#for i in range(1, 2):
    #params.kpi = i
    #sim = simulate(params=params, inData = inData,  kpi_veh=kpi_veh_pricing,kpi_pax=kpi_pax_pricing, logger_level=logging.WARNING)
   # sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    #sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv'.format(i))
   # sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv'.format(i))
   

In [501]:
#sim.vehs[2].rdf 

In [502]:
#pd.DataFrame(sim.vehs[2].myrides)

## Script 